Welcome to the Aiven Clickhouse challenge!

With the launch of Aiven for Clickhouse We finally have the "C" for our KFC stack, the best combination of tools for realtime aleart and historic analytics.

The stack untilises Apache Kafka, Apache Flink, and Clickhouse to build up a robust, scleable architecture for getting the most from your data, whether it be batched ETL or realtime sensors.

For this challenge, We'll be using realtime data from [digitransit](https://digitransit.fi/en/developers/apis/4-realtime-api/) to plot maps and find new insights.

Digitransit provides reatime bus informantion and positioning information for busses in Helsinki, We've saved out a few weeks worth of data as a CSV to experiemnt with.

The goal is to find the best bus routes between the different KFC restaurants in Helsinki. To enter, create a trial account on Aiven to generate and to process the data, Find the routes, plot them on a map and tag us / DM us on twitter with a screen shot of a map with these routes plotted on it.  

The prize for this is a limited edition Hat + The knowledge that you can indeed navigate public transport with the right SQL statement.

This notbook domes with a few starter queries and a mapping library to get you started. none of the queries here needed but they should help you get started. 

Please see the Aiven [docs](https://docs.aiven.io/docs/products/clickhouse/getting-started.html) for Clickhouse info on how to get step up and the main clickhouse [docs](https://clickhouse.com/docs/en/sql-reference) to for advanced SQL information 

For more help/ questions ping us on twitter [@aiven_io](https://twitter.com/aiven_io), or me [@Bengamble7](https://twitter.com/BenGamble7) and we'll provide hints and even more terrible puns and clickhouse, or any of our 11 ~~secrect herbs and spices~~ products




In [ ]:

from clickhouse_driver import Client
client = Client('YOUR-CLICKHOUSE-URL',
                user='avnadmin',
                password='YOUR-PASSWORD',
                secure=True,
                port='YOUR-PORT',
                )

#to confirm its all wowrking
client.execute('SHOW DATABASES')

In [ ]:

#So there is a lot of data here 
count_query = 'select count() from default.DigitransitINPUT'

out = client.execute(count_query)

out

In [ ]:
#This is a very narrow query but it shows what you can do (limit is quite important)
first_query = 'select lat, long from default.DigitransitINPUT where (veh = 1344) AND (spd > 15.0) limit 500'

out = client.execute(first_query)

len(out)


In [ ]:
next_query = 'select lat, long from default.DigitransitINPUT where  (spd > 45.0) limit 500'

out = client.execute(next_query)

len(out)


For mapping I've selected [Folium](http://python-visualization.github.io/folium/)

Its pretty, fast and flexible

Here, it's set to the center of the Helsinki Metro area

In [ ]:
import folium
m = folium.Map(location=[60.40, 25.1067], zoom_start=11)

for point in out:
    try:
        folium.Marker(point).add_to(m)
    except:
        pass
    
m

In [ ]:
out = client.execute('select max (spd) from default.DigitransitINPUT')
out


This is an interesting one as it lets us map colors to speed :)


In [ ]:
import math

marker_colors = ['red', 'darkred',  'lightred', 'orange', 'beige',
                 'green', 'darkgreen', 'lightgreen',
                 'blue', 'darkblue', 'cadetblue', 'lightblue',
                 'purple',  'darkpurple', 'pink',
                 'white', 'gray', 'lightgray', 'black']



def colorise(speed_in: float)-> str:
    return marker_colors[math.floor(speed_in/50)]

In [ ]:
def make_map(my_points):
    map = folium.Map(location=[60.40, 25.1067], zoom_start=11)

    for point in my_points:
        try:
            folium.Marker(location=[point[0], point[1]],  icon=folium.Icon(color=colorise(point[2])) ).add_to(map)
        except:
            pass

    return map

In [ ]:
next_query = 'select lat, long, spd from default.DigitransitINPUT limit 500'
out = client.execute(next_query)

map = folium.Map(location=[60.40, 25.1067], zoom_start=11)
color_str =''
for point in out:
    try:
        folium.Marker(location=[point[0], point[1]], color = colorise(point[2])).add_to(map)

    except:
        pass

        
map